In [1]:
import pandas as pd
import json
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

In [2]:
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    # Initialize an empty list to hold flattened data
    flattened_data = []

    # Iterate over the outer list
    for item in data:
        # Check if the item is a dictionary
        if isinstance(item, dict):
            flattened_data.append(item)
        # If the item is a list, iterate over the inner list
        elif isinstance(item, list):
            for sub_item in item:
                flattened_data.append(sub_item)

    # Convert flattened data to Pandas DataFrame
    return pd.DataFrame(flattened_data)    

In [3]:
# Load train flag data
train_flag_path = 'train_flag.csv'
train_flag_df = pd.read_csv(train_flag_path)

In [4]:
train_flag_df

,uid,NAME_CONTRACT_TYPE,TARGET
0,XDA69787158,Cash loans,0
1,BSE47789733,Cash loans,0
2,NTJ92213825,Cash loans,0
3,TCQ47571695,Cash loans,0
4,WJZ68772744,Cash loans,0
...,...,...,...
261378,BAO14050568,Cash loans,0
261379,NDO00278120,Cash loans,0
261380,LDY78194712,Cash loans,0
261381,EVH98380736,Revolving loans,0


In [5]:
# Load accounts data
accounts_data_path = 'accounts_data_train.json'
accounts_data_df = load_json_data(accounts_data_path)

In [6]:
accounts_data_df

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,272745.000,0.0,2018-09-22,2020-02-22,0000000000000000000000100000000000000000000000...,AAA09044550
1,Consumer credit,4500.000,0.0,2018-03-08,2019-07-25,000000000000000014044000000000000000000000000000,AAA09044550
2,Credit card,80996.445,0.0,2020-06-29,NaN,000000000000000000,AAA10545297
3,Consumer credit,43771.500,0.0,2020-06-09,2020-09-09,000000000,AAA14112888
4,Credit card,10480.500,0.0,2014-09-10,NaN,0000000000000000000000000000000000000000000000...,AAA20326915
...,...,...,...,...,...,...,...
1245305,Consumer credit,100116.000,0.0,2017-05-12,2017-06-15,000,ZZZ93377613
1245306,Consumer credit,34839.000,0.0,2016-11-11,2017-12-17,000000000000000000000000000000000000000,ZZZ93377613
1245307,Consumer credit,18589.500,0.0,2015-12-10,2016-05-12,000000000000000,ZZZ93377613
1245308,Consumer credit,24997.500,0.0,2020-09-19,NaN,000000000,ZZZ93377613


In [7]:
# Load enquiry data
enquiry_data_path = 'enquiry_data_train.json'
enquiry_data_df = load_json_data(enquiry_data_path)

In [8]:
enquiry_data_df

,enquiry_type,enquiry_amt,enquiry_date,uid
0,Interbank credit,168839,2020-11-08,AAA08065248
1,Mobile operator loan,268392,2020-09-20,AAA08065248
2,Mobile operator loan,36082,2020-06-19,AAA08065248
3,Interbank credit,180467,2019-10-22,AAA08065248
4,Cash loan (non-earmarked),227459,2020-05-24,AAA08065248
...,...,...,...,...
1909921,Loan for the purchase of equipment,401073,2020-02-23,ZZZ93377613
1909922,Consumer credit,224844,2018-06-03,ZZZ93377613
1909923,Loan for purchase of shares (margin lending),416642,2017-10-15,ZZZ93377613
1909924,Loan for purchase of shares (margin lending),22000,2020-08-22,ZZZ93377613


In [9]:
# Convert date columns to datetime format
accounts_data_df['open_date'] = pd.to_datetime(accounts_data_df['open_date'])
accounts_data_df['closed_date'] = pd.to_datetime(accounts_data_df['closed_date'])
enquiry_data_df['enquiry_date'] = pd.to_datetime(enquiry_data_df['enquiry_date'])

In [10]:
# Feature Engineering for Accounts Data
accounts_data_df['loan_duration'] = (accounts_data_df['closed_date'] - accounts_data_df['open_date']).dt.days
accounts_features = accounts_data_df.groupby('uid').agg({
    'loan_amount': ['mean', 'sum', 'max'],
    'amount_overdue': ['mean', 'sum', 'max'],
    'loan_duration': ['mean', 'max']
}).reset_index()
accounts_features.columns = ['uid', 'loan_amount_mean', 'loan_amount_sum', 'loan_amount_max',
                             'amount_overdue_mean', 'amount_overdue_sum', 'amount_overdue_max',
                             'loan_duration_mean', 'loan_duration_max']

# Feature Engineering for Enquiry Data
enquiry_features = enquiry_data_df.groupby('uid').agg({
    'enquiry_amt': ['mean', 'sum', 'max'],
    'enquiry_date': ['count']
}).reset_index()
enquiry_features.columns = ['uid', 'enquiry_amt_mean', 'enquiry_amt_sum', 'enquiry_amt_max', 'enquiry_count']

In [11]:
# Merge the datasets on the 'uid' column
merged_df = train_flag_df.merge(accounts_data_df, on='uid', how='left')
merged_df = merged_df.merge(enquiry_data_df, on='uid', how='left')

In [12]:
merged_df

,uid,NAME_CONTRACT_TYPE,TARGET,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,loan_duration,enquiry_type,enquiry_amt,enquiry_date
0,XDA69787158,Cash loans,0,Credit card,450000.0,0.0,2017-10-01,NaT,0000000000000000000000000000000000000000000000...,NaN,Microloan,166000,2020-08-07
1,XDA69787158,Cash loans,0,Credit card,450000.0,0.0,2017-10-01,NaT,0000000000000000000000000000000000000000000000...,NaN,Real estate loan,105000,2020-08-02
2,BSE47789733,Cash loans,0,Consumer credit,110155.5,0.0,2018-09-26,2018-10-26,000,30.0,Credit card,151000,2020-02-11
3,BSE47789733,Cash loans,0,Consumer credit,110155.5,0.0,2018-09-26,2018-10-26,000,30.0,Car loan,132000,2020-09-25
4,BSE47789733,Cash loans,0,Consumer credit,110155.5,0.0,2018-09-26,2018-10-26,000,30.0,Cash loans,150000,2020-11-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9375750,EVH98380736,Revolving loans,0,Consumer credit,1105380.0,0.0,2019-12-10,NaT,000000000000000000000000000000000000,NaN,Revolving loans,206231,2020-12-11
9375751,EVH98380736,Revolving loans,0,Consumer credit,472950.0,0.0,2020-02-11,NaT,000000000000000000000000000000,NaN,Revolving loans,206231,2020-12-11
9375752,AUB40959272,Cash loans,1,Consumer credit,94185.0,0.0,2019-10-22,2020-09-03,000000000000000000000000000000,317.0,Cash loans,145000,2020-09-30
9375753,AUB40959272,Cash loans,1,Consumer credit,94185.0,0.0,2019-10-22,2020-09-03,000000000000000000000000000000,317.0,Cash loans,18000,2020-06-16


In [13]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

uid                          0
NAME_CONTRACT_TYPE           0
TARGET                       0
credit_type             273136
loan_amount             273162
amount_overdue          273136
open_date               273136
closed_date            3658746
payment_hist_string     273136
loan_duration          3658746
enquiry_type                 0
enquiry_amt                  0
enquiry_date                 0
dtype: int64


In [14]:
# Fill missing values in numeric columns with mean or median
merged_df['loan_amount'].fillna(merged_df['loan_amount'].mean(), inplace=True)
merged_df['amount_overdue'].fillna(merged_df['amount_overdue'].median(), inplace=True)
merged_df['loan_duration'] = merged_df['loan_duration'].fillna(merged_df['loan_duration'].mean())

# Fill missing values in categorical columns with mode
merged_df['credit_type'].fillna(merged_df['credit_type'].mode()[0], inplace=True)

# Fill missing values in date columns with forward fill (or backward fill as needed)
merged_df['open_date'].fillna(method='ffill', inplace=True)
merged_df['closed_date'].fillna(method='bfill', inplace=True)

# Fill missing values in string columns with mode or a specific string
merged_df['payment_hist_string'].fillna('', inplace=True)

# Verify that all null values are filled
print(merged_df.isnull().sum())

uid                    0
NAME_CONTRACT_TYPE     0
TARGET                 0
credit_type            0
loan_amount            0
amount_overdue         0
open_date              0
closed_date            0
payment_hist_string    0
loan_duration          0
enquiry_type           0
enquiry_amt            0
enquiry_date           0
dtype: int64


In [15]:
print(merged_df.dtypes)

uid                            object
NAME_CONTRACT_TYPE             object
TARGET                          int64
credit_type                    object
loan_amount                   float64
amount_overdue                float64
open_date              datetime64[ns]
closed_date            datetime64[ns]
payment_hist_string            object
loan_duration                 float64
enquiry_type                   object
enquiry_amt                     int64
enquiry_date           datetime64[ns]
dtype: object


In [16]:
# Encode categorical variables using sparse format
categorical_cols = ['NAME_CONTRACT_TYPE', 'credit_type', 'enquiry_type']
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
encoded_cats_sparse = encoder.fit_transform(merged_df[categorical_cols])

# Convert the sparse matrix to a DataFrame
encoded_cats = pd.DataFrame.sparse.from_spmatrix(encoded_cats_sparse, columns=encoder.get_feature_names_out(categorical_cols))

# Concatenate the encoded columns with the original dataframe
merged_df = pd.concat([merged_df.reset_index(drop=True), encoded_cats.reset_index(drop=True)], axis=1)

# Drop unused columns
merged_df.drop(columns=categorical_cols + ['open_date', 'closed_date', 'enquiry_date', 'payment_hist_string'], inplace=True)

# Verify the dataframe
print(merged_df.info())

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9375755 entries, 0 to 9375754
Data columns (total 40 columns):
 #   Column                                                     Dtype             
---  ------                                                     -----             
 0   uid                                                        object            
 1   TARGET                                                     int64             
 2   loan_amount                                                float64           
 3   amount_overdue                                             float64           
 4   loan_duration                                              float64           
 5   enquiry_amt                                                int64             
 6   NAME_CONTRACT_TYPE_Cash loans                              Sparse[float64, 0]
 7   NAME_CONTRACT_TYPE_Revolving loans                         Sparse[float64, 0]
 8   credit_type_Another type of loan                    

In [17]:
# Split the data into features and target
X = merged_df.drop(columns=['TARGET', 'uid'])
y = merged_df['TARGET']

In [44]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_val_pred = model.predict_proba(X_val)[:, 1]
    roc_auc = roc_auc_score(y_val, y_val_pred)
    print(f'{name} Validation ROC AUC Score: {roc_auc:.4f}')
    
    if roc_auc > best_roc_auc:
        best_roc_auc = roc_auc
        best_model = model

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Logistic Regression Validation ROC AUC Score: 0.5162


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Decision Tree Validation ROC AUC Score: 0.7713


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\utils\validation.py:787: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


KeyboardInterrupt: 

In [18]:
# Test Data

In [19]:
test_flag_path = 'test_flag.csv'
accounts_data_test_path = 'accounts_data_train.json'
enquiry_data_test_path = 'enquiry_data_test.json'

test_flag = pd.read_csv(test_flag_path)
accounts_data_test = load_json_data(accounts_data_test_path)
enquiry_data_test = load_json_data(enquiry_data_test_path)

In [20]:
# Convert date columns to datetime format
accounts_data_test['open_date'] = pd.to_datetime(accounts_data_test['open_date'])
accounts_data_test['closed_date'] = pd.to_datetime(accounts_data_test['closed_date'])
enquiry_data_test['enquiry_date'] = pd.to_datetime(enquiry_data_test['enquiry_date'])

In [21]:
# Feature Engineering for Accounts Data
accounts_data_test['loan_duration'] = (accounts_data_test['closed_date'] - accounts_data_test['open_date']).dt.days
accounts_features = accounts_data_test.groupby('uid').agg({
    'loan_amount': ['mean', 'sum', 'max'],
    'amount_overdue': ['mean', 'sum', 'max'],
    'loan_duration': ['mean', 'max']
}).reset_index()
accounts_features.columns = ['uid', 'loan_amount_mean', 'loan_amount_sum', 'loan_amount_max',
                             'amount_overdue_mean', 'amount_overdue_sum', 'amount_overdue_max',
                             'loan_duration_mean', 'loan_duration_max']

# Feature Engineering for Enquiry Data
enquiry_features = enquiry_data_test.groupby('uid').agg({
    'enquiry_amt': ['mean', 'sum', 'max'],
    'enquiry_date': ['count']
}).reset_index()
enquiry_features.columns = ['uid', 'enquiry_amt_mean', 'enquiry_amt_sum', 'enquiry_amt_max', 'enquiry_count']

In [27]:
# Merge the datasets on the 'uid' column
test_merged_df = train_flag_df.merge(accounts_data_df, on='uid', how='left')
test_merged_df = test_merged_df.merge(enquiry_data_df, on='uid', how='left')

In [28]:
nan_counts = test_merged_df.isna().sum()
print(nan_counts)

uid                          0
NAME_CONTRACT_TYPE           0
TARGET                       0
credit_type             273136
loan_amount             273162
amount_overdue          273136
open_date               273136
closed_date            3658746
payment_hist_string     273136
loan_duration          3658746
enquiry_type                 0
enquiry_amt                  0
enquiry_date                 0
dtype: int64


In [29]:
# Fill missing values in numeric columns with mean or median
test_merged_df['loan_amount'].fillna(test_merged_df['loan_amount'].mean(), inplace=True)
test_merged_df['amount_overdue'].fillna(test_merged_df['amount_overdue'].median(), inplace=True)
test_merged_df['loan_duration'] = test_merged_df['loan_duration'].fillna(test_merged_df['loan_duration'].mean())

# Fill missing values in categorical columns with mode
test_merged_df['credit_type'].fillna(test_merged_df['credit_type'].mode()[0], inplace=True)

# Fill missing values in date columns with forward fill (or backward fill as needed)
test_merged_df['open_date'].fillna(method='ffill', inplace=True)
test_merged_df['closed_date'].fillna(method='bfill', inplace=True)

# Fill missing values in string columns with mode or a specific string
test_merged_df['payment_hist_string'].fillna('', inplace=True)

# Verify that all null values are filled
print(test_merged_df.isnull().sum())

uid                    0
NAME_CONTRACT_TYPE     0
TARGET                 0
credit_type            0
loan_amount            0
amount_overdue         0
open_date              0
closed_date            0
payment_hist_string    0
loan_duration          0
enquiry_type           0
enquiry_amt            0
enquiry_date           0
dtype: int64


In [31]:
# Encode categorical variables using sparse format
categorical_cols = ['NAME_CONTRACT_TYPE', 'credit_type', 'enquiry_type']
encoder = OneHotEncoder(sparse=True, handle_unknown='ignore')
encoded_cats_sparse = encoder.fit_transform(test_merged_df[categorical_cols])

# Convert the sparse matrix to a DataFrame
encoded_cats = pd.DataFrame.sparse.from_spmatrix(encoded_cats_sparse, columns=encoder.get_feature_names_out(categorical_cols))

# Concatenate the encoded columns with the original dataframe
test_merged_df = pd.concat([test_merged_df.reset_index(drop=True), encoded_cats.reset_index(drop=True)], axis=1)

# Drop unused columns
test_merged_df.drop(columns=categorical_cols + ['open_date', 'closed_date', 'enquiry_date', 'payment_hist_string'], inplace=True)

# Verify the dataframe
print(test_merged_df.info())

C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9375755 entries, 0 to 9375754
Data columns (total 40 columns):
 #   Column                                                     Dtype             
---  ------                                                     -----             
 0   uid                                                        object            
 1   TARGET                                                     int64             
 2   loan_amount                                                float64           
 3   amount_overdue                                             float64           
 4   loan_duration                                              float64           
 5   enquiry_amt                                                int64             
 6   NAME_CONTRACT_TYPE_Cash loans                              Sparse[float64, 0]
 7   NAME_CONTRACT_TYPE_Revolving loans                         Sparse[float64, 0]
 8   credit_type_Another type of loan                    

In [ ]:
# Prepare the test features
X_test = test_merged_df.drop(columns=['uid'])

# Make predictions on test data using the best model
test_preds = best_model.predict_proba(X_test)[:, 1]

# Prepare the submission file
submission_df = pd.DataFrame({
    'uid': test_merged_df['uid'],
    'TARGET': test_preds
})

submission_path = 'Monsoon_Credit_Assignment.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Submission file saved to {submission_path}")